### **Curating body_breast1.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12887263.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 2097 × 17194
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.05017601, -0.17139998, -0.17394103, ..., -0.24901268,
        -0.22266223, -0.1450203 ],
       [-0.05017601, -0.17139998, -0.17394103, ..., -0.24901268,
        -0.22266223, -0.1450203 ],
       [-0.05017601, -0.17139998, -0.17394103, ..., -0.24901268,
        -0.22266223, -0.1450203 ],
       ...,
       [-0.05017601, -0.17139998, -0.17394103, ..., -0.24901268,
        -0.22266223,  5.4244795 ],
       [-0.05017601, -0.17139998, -0.17394103, ..., -0.24901268,
        -0.22266223, -0.1450203 ],
       [-0.05017601, -0.17139998, -0.17394103, ..., -0.24901268,
        -0.22266223, -0.1450203 ]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<2097x17194 sparse matrix of type '<class 'numpy.float32'>'
	with 36055818 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.05017601
  (0, 1)	-0.17139998
  (0, 2)	-0.17394103
  (0, 3)	-0.044478133
  (0, 4)	-0.08837432
  (0, 5)	-0.046985712
  (0, 6)	-0.049258687
  (0, 7)	-0.36668572
  (0, 8)	-0.16194707
  (0, 9)	-0.2501982
  (0, 10)	-0.09103287
  (0, 11)	0.99248356
  (0, 12)	-0.41153118
  (0, 13)	-0.27309093
  (0, 14)	-0.08071196
  (0, 15)	-0.24728023
  (0, 16)	-0.13319968
  (0, 17)	-0.06282209
  (0, 18)	-0.27848032
  (0, 19)	-0.17020345
  (0, 20)	0.99190015
  (0, 21)	3.8041627
  (0, 22)	-0.08889841
  (0, 23)	-0.36241958
  (0, 24)	-0.20464246
  :	:
  (2096, 17169)	-0.06322248
  (2096, 17170)	-0.21495378
  (2096, 17171)	-0.060591083
  (2096, 17172)	0.39236692
  (2096, 17173)	0.305472
  (2096, 17174)	-0.9287654
  (2096, 17175)	-1.3384545
  (2096, 17176)	-0.45341316
  (2096, 17177)	0.24071132
  (2096, 17178)	-0.3952289
  (2096, 17179)	-0.4000277
  (2096, 17180)	-0.7369301
  (2096, 17181)	-0.15995501
  (2096, 17182)	-1.2408016
  (2096, 17183)	-0.12769955
  (2096, 17184)	0.115392186
  (2096, 17185)	-0

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12887263/filtered_feature_bc_matrix.h5')

/home/jovyan/my-conda-envs/shi_env/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 2097 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<2097x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 3090796 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 29)	1.0
  (0, 44)	1.0
  (0, 45)	1.0
  (0, 52)	1.0
  (0, 58)	1.0
  (0, 71)	1.0
  (0, 73)	1.0
  (0, 79)	2.0
  (0, 84)	2.0
  (0, 86)	1.0
  (0, 97)	1.0
  (0, 104)	1.0
  (0, 113)	1.0
  (0, 134)	2.0
  (0, 173)	1.0
  (0, 194)	1.0
  (0, 219)	2.0
  (0, 228)	1.0
  (0, 237)	1.0
  (0, 247)	1.0
  (0, 286)	1.0
  (0, 300)	1.0
  (0, 336)	2.0
  (0, 362)	1.0
  (0, 407)	1.0
  :	:
  (2096, 35598)	1.0
  (2096, 35605)	1.0
  (2096, 35624)	1.0
  (2096, 35630)	2.0
  (2096, 35734)	1.0
  (2096, 35764)	1.0
  (2096, 35839)	2.0
  (2096, 35880)	1.0
  (2096, 35942)	1.0
  (2096, 36079)	1.0
  (2096, 36085)	1.0
  (2096, 36097)	2.0
  (2096, 36136)	1.0
  (2096, 36398)	1.0
  (2096, 36401)	2.0
  (2096, 36406)	1.0
  (2096, 36559)	3.0
  (2096, 36560)	2.0
  (2096, 36561)	1.0
  (2096, 36562)	1.0
  (2096, 36564)	3.0
  (2096, 36565)	3.0
  (2096, 36566)	1.0
  (2096, 36568)	3.0
  (2096, 36571)	2.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
LINC01409
LINC01128
LINC00115
FAM41C
...
AL592183.1
AC240274.1
AC004556.3
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
LINC01409,LINC01409,ENSG00000237491
LINC01128,LINC01128,ENSG00000228794
LINC00115,LINC00115,ENSG00000225880
FAM41C,FAM41C,ENSG00000230368
...,...,...
AL592183.1,AL592183.1,ENSG00000273748
AC240274.1,AC240274.1,ENSG00000271254
AC004556.3,AC004556.3,ENSG00000276345
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

14

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000228794,LINC01128,ENSG00000228794
ENSG00000225880,LINC00115,ENSG00000225880
ENSG00000230368,FAM41C,ENSG00000230368
...,...,...
ENSG00000273748,AL592183.1,ENSG00000273748
ENSG00000271254,AC240274.1,ENSG00000271254
ENSG00000276345,AC004556.3,ENSG00000276345


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000237491
ENSG00000228794
ENSG00000225880
ENSG00000230368
...
ENSG00000273748
ENSG00000271254
ENSG00000276345


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/gene_info/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

116184

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

17153

In [57]:
len(var_to_keep_araw)

36503

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000237491
ENSG00000228794
ENSG00000225880
ENSG00000230368
...
ENSG00000273748
ENSG00000271254
ENSG00000276345


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

19350

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.046429,0.002225,0.041110,0.029031,0.004588,5,0.257870,0.020301,0.058313,0.028732
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.028695,0.000623,0.007338,0.138582,0.002524,2,0.320447,0.043960,0.019942,0.004002
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.018232,0.001158,0.046217,0.031908,0.002216,1,0.258645,0.009907,0.036645,0.005254
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.022250,0.000656,0.007977,0.024529,0.002249,2,0.260169,0.016240,0.017285,0.006448
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.041565,0.001375,0.009207,0.002953,0.007286,2,0.805861,0.074709,0.005254,0.003218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.011424,0.000424,0.015090,4.324036,0.010564,7,0.144531,0.014687,0.095812,0.026276
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.014052,0.000862,0.003107,0.006652,0.203137,0,2.643321,0.250544,0.009137,0.006234
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,0.038001,0.001381,0.412067,0.527707,1.511992,6,0.407080,5.848951,3.239772,0.004490


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.002225,0.041110,0.029031,0.004588,5,0.257870,0.020301,0.058313,0.028732,EFO:0010961
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.000623,0.007338,0.138582,0.002524,2,0.320447,0.043960,0.019942,0.004002,EFO:0010961
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.001158,0.046217,0.031908,0.002216,1,0.258645,0.009907,0.036645,0.005254,EFO:0010961
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.000656,0.007977,0.024529,0.002249,2,0.260169,0.016240,0.017285,0.006448,EFO:0010961
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.001375,0.009207,0.002953,0.007286,2,0.805861,0.074709,0.005254,0.003218,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.000424,0.015090,4.324036,0.010564,7,0.144531,0.014687,0.095812,0.026276,EFO:0010961
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.000862,0.003107,0.006652,0.203137,0,2.643321,0.250544,0.009137,0.006234,EFO:0010961
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,0.001381,0.412067,0.527707,1.511992,6,0.407080,5.848951,3.239772,0.004490,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.029031,0.004588,5,0.257870,0.020301,0.058313,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.138582,0.002524,2,0.320447,0.043960,0.019942,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.031908,0.002216,1,0.258645,0.009907,0.036645,0.005254,EFO:0010961,c2l_VEC,2.114354
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.024529,0.002249,2,0.260169,0.016240,0.017285,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.002953,0.007286,2,0.805861,0.074709,0.005254,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,4.324036,0.010564,7,0.144531,0.014687,0.095812,0.026276,EFO:0010961,c2l_SMC,4.324036
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.006652,0.203137,0,2.643321,0.250544,0.009137,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,0.527707,1.511992,6,0.407080,5.848951,3.239772,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [ ]:
list(adata.obs['cell_type_ontology_term_id'].unique())

In [94]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.004588,5,0.257870,0.020301,0.058313,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.002524,2,0.320447,0.043960,0.019942,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.002216,1,0.258645,0.009907,0.036645,0.005254,EFO:0010961,c2l_VEC,2.114354,CL:0002139
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.002249,2,0.260169,0.016240,0.017285,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.007286,2,0.805861,0.074709,0.005254,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.010564,7,0.144531,0.014687,0.095812,0.026276,EFO:0010961,c2l_SMC,4.324036,CL:0000192
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.203137,0,2.643321,0.250544,0.009137,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,1.511992,6,0.407080,5.848951,3.239772,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187


In [95]:
araw.obs

""
AAACACCAATAACTGC-1
AAACAGCTTTCAGAAG-1
AAACAGGGTCTATATT-1
AAACATGGTGAGAGGA-1
AAACCGGGTAGGTACC-1
...
TTGTTCAGTGTGCTAC-1
TTGTTCTAGATACGCT-1
TTGTTTCACATCCAGG-1
TTGTTTCATTAGTCTA-1


#### **donor_id**

In [96]:
#identify the column in adata.obs which provides donor information

In [97]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [98]:
# add the donor_id column

In [99]:
adata.obs['donor_id'] = ['WSSKNKCLsp12887263'] * len(adata.obs)

In [100]:
# change datatype of the column

In [101]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [102]:
# view unique values of donor_id column

In [103]:
list(adata.obs['donor_id'].unique())

['WSSKNKCLsp12887263']

In [104]:
#view obs

In [105]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,5,0.257870,0.020301,0.058313,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,2,0.320447,0.043960,0.019942,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,1,0.258645,0.009907,0.036645,0.005254,EFO:0010961,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,2,0.260169,0.016240,0.017285,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,2,0.805861,0.074709,0.005254,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,7,0.144531,0.014687,0.095812,0.026276,EFO:0010961,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0,2.643321,0.250544,0.009137,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,6,0.407080,5.848951,3.239772,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263


In [106]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [107]:
# identify the column in adata which corresponds to age

In [108]:
# add the development_stage_ontology_term_id column

In [109]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [110]:
mapping = dict(zip(suppl_info['donor'], suppl_info['development_stage_ontology_term_id']))

In [111]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [112]:
# change datatype of the column

In [113]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [114]:
# view unique values of development_stage_ontology_term_id column

In [115]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000136']

In [116]:
# view adata.obs

In [117]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.257870,0.020301,0.058313,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.320447,0.043960,0.019942,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.258645,0.009907,0.036645,0.005254,EFO:0010961,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.260169,0.016240,0.017285,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.805861,0.074709,0.005254,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.144531,0.014687,0.095812,0.026276,EFO:0010961,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,2.643321,0.250544,0.009137,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,0.407080,5.848951,3.239772,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136


#### **disease_ontology_term_id**

In [118]:
# Assign normal since all are healthy patients

In [119]:
# add the disease_ontology_term_id column

In [120]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [121]:
mapping = dict(zip(suppl_info['donor'], suppl_info['disease_ontology_term_id']))

In [122]:
adata.obs['disease_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [123]:
#change data type of column

In [124]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [125]:
# view obs

In [126]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.020301,0.058313,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.043960,0.019942,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.009907,0.036645,0.005254,EFO:0010961,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.016240,0.017285,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.074709,0.005254,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.014687,0.095812,0.026276,EFO:0010961,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.250544,0.009137,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,5.848951,3.239772,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461


#### **is_primary_data**

In [127]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [128]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.058313,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.019942,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.036645,0.005254,EFO:0010961,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.017285,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.005254,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.095812,0.026276,EFO:0010961,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.009137,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,3.239772,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True


In [129]:
#change data type of column

In [130]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [131]:
# assign organism id 

In [132]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [133]:
#change data type of column

In [134]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [135]:
# view obs

In [136]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,0.028732,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,0.004002,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,0.005254,EFO:0010961,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,0.006448,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,0.003218,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,0.026276,EFO:0010961,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,0.006234,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,0.004490,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [137]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [138]:
# change data type

In [139]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [140]:
# view obs

In [141]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,EFO:0010961,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,EFO:0010961,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,EFO:0010961,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown


In [142]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [143]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [144]:
mapping = dict(zip(suppl_info['donor'], suppl_info['sex_ontology_term_id']))

In [145]:
adata.obs['sex_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [146]:
# change data type

In [147]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [148]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,c2l_SFRP2+ fibroblasts,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,c2l_SFRP2+ fibroblasts,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,c2l_VEC,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,c2l_SFRP2+ fibroblasts,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,c2l_POSTN+ fibroblasts,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,c2l_SMC,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,c2l_SFRP2+ fibroblasts,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,c2l_Basal keratinocytes,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383


#### **suspension_type**

In [149]:
# since visium suspension type is 'na'

In [150]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [151]:
# change data type

In [152]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [153]:
# view obs

In [154]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,11.212178,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,2.008300,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,2.114354,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,8.841479,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,1.873764,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,4.324036,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,3.360085,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,5.848951,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na


#### **tissue_ontology_term_id**

In [155]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [156]:
mapping = dict(zip(suppl_info['donor'], suppl_info['tissue_ontology_term_id']))

In [157]:
adata.obs['tissue_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [158]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [159]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:8480030']

In [160]:
# view obs

In [161]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,1467,2951,103,3.490342,303,10.267706,AAACACCAATAACTGC-1,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACAGCTTTCAGAAG-1,1,43,9,564,733,23,3.137790,61,8.321964,AAACAGCTTTCAGAAG-1,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACAGGGTCTATATT-1,1,47,13,529,681,30,4.405286,81,11.894273,AAACAGGGTCTATATT-1,...,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACATGGTGAGAGGA-1,1,62,0,923,1613,79,4.897706,137,8.493490,AAACATGGTGAGAGGA-1,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACCGGGTAGGTACC-1,1,42,28,600,851,31,3.642773,104,12.220917,AAACCGGGTAGGTACC-1,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,2321,5445,208,3.820019,544,9.990817,TTGTTCAGTGTGCTAC-1,...,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
TTGTTCTAGATACGCT-1,1,21,3,1138,1903,54,2.837625,245,12.874410,TTGTTCTAGATACGCT-1,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
TTGTTTCACATCCAGG-1,1,58,42,3455,8744,638,7.296432,1065,12.179781,TTGTTTCACATCCAGG-1,...,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030


In [162]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [163]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [164]:
adata.obsm

AxisArrays with keys: X_spatial

In [165]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [166]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([342.79993  ,  77.75388  ,  54.15468  ,  45.208687 ,  28.555157 ,
        22.9152   ,  20.204771 ,  19.883926 ,  19.20316  ,  18.30488  ,
        17.596727 ,  16.538595 ,  16.20171  ,  15.837937 ,  15.816287 ,
        15.588081 ,  15.542452 ,  15.452355 ,  15.437598 ,  15.419341 ,
        15.373127 ,  15.312593 ,  15.279792 ,  15.255082 ,  15.215959 ,
        15.165711 ,  15.161683 ,  15.154871 ,  15.118574 ,  15.074039 ,
        15.042105 ,  15.004319 ,  14.99773  ,  14.9643135,  14.961903 ,
        14.923525 ,  14.9148245,  14.875512 ,  14.861321 ,  14.855446 ,
        14.813731 ,  14.786464 ,  14.761067 

In [167]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the breast stained with H&E'

In [168]:
adata.uns['title'] = 'Visium spatial - body_breast1'

In [169]:
adata.uns['default_embedding'] = 'X_spatial'

In [170]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [171]:
adata

AnnData object with n_obs × n_vars = 2097 × 36503
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_o

In [172]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [173]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [174]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [175]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,1467,2951,103,3.490342,303,10.267706,0.014459,0.029487,0.210160,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACAGCTTTCAGAAG-1,1,564,733,23,3.137790,61,8.321964,0.002157,0.005207,0.009265,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACAGGGTCTATATT-1,1,529,681,30,4.405286,81,11.894273,0.012597,0.026243,0.648415,...,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACATGGTGAGAGGA-1,1,923,1613,79,4.897706,137,8.493490,0.007827,0.012632,0.032212,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACCGGGTAGGTACC-1,1,600,851,31,3.642773,104,12.220917,0.009282,0.023874,0.014695,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,2321,5445,208,3.820019,544,9.990817,0.001428,0.005832,0.020562,...,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
TTGTTCTAGATACGCT-1,1,1138,1903,54,2.837625,245,12.874410,0.003077,0.004258,0.011160,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
TTGTTTCACATCCAGG-1,1,3455,8744,638,7.296432,1065,12.179781,0.018507,0.010640,1.166922,...,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030


In [176]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [177]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [178]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,1467,2951,103,3.490342,303,10.267706,0.014459,0.029487,0.210160,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACAGCTTTCAGAAG-1,1,564,733,23,3.137790,61,8.321964,0.002157,0.005207,0.009265,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACAGGGTCTATATT-1,1,529,681,30,4.405286,81,11.894273,0.012597,0.026243,0.648415,...,CL:0002139,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACATGGTGAGAGGA-1,1,923,1613,79,4.897706,137,8.493490,0.007827,0.012632,0.032212,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
AAACCGGGTAGGTACC-1,1,600,851,31,3.642773,104,12.220917,0.009282,0.023874,0.014695,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,2321,5445,208,3.820019,544,9.990817,0.001428,0.005832,0.020562,...,CL:0000192,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
TTGTTCTAGATACGCT-1,1,1138,1903,54,2.837625,245,12.874410,0.003077,0.004258,0.011160,...,CL:0000057,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030
TTGTTTCACATCCAGG-1,1,3455,8744,638,7.296432,1065,12.179781,0.018507,0.010640,1.166922,...,CL:0002187,WSSKNKCLsp12887263,HsapDv:0000136,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:8480030


In [179]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [180]:
#check the format of expression matrix

In [181]:
adata.X

<2097x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 35969841 stored elements in Compressed Sparse Row format>

In [182]:
araw.X

<2097x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 3089596 stored elements in Compressed Sparse Row format>

In [183]:
#Copy raw counts to adata.raw

In [184]:
adata.raw = araw

In [185]:
#write the curated object to final_objects folder

In [186]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/body_breast1.h5ad', compression = 'gzip')